# Buld the Neural Network

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

/home/dbsejrgus226/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dbsejrgus226/miniconda3/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/dbsejrgus226/miniconda3/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Get Device for Training


In [3]:
device = ('cuda'
          if torch.cuda.is_available()
          else 'mps'
          if torch.backends.mps.is_available()
          else 'cpu')

print(f"Using {device} device")

Using cpu device


## Define the Class
 - ``nn.Module``의 subclassing으로 neural network를 정의
 - ``__init``으로 신경망 layer 초기화

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


## Model Layers


In [9]:
input_image = torch.rand(3,28,28) # rand(3,28,28)이란 28x28 크기의 3개 이미지로 구성된 random image
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten
``nn.Flatten``으로 layer를 초기화하여 각 2d 28x28 이미지를 784 pixel 연속 배열로 변환

In [10]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear
``nn.Linear`` 저장된 가중치와 편향을 사용하여 input에 선형 변환을 적용하는 모듈

In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU
 - 모델의 입력과 출력 사이 복잡한 mapping을 생성
 - nn.Linear(선형 변환) 후 nn.ReLU(비선형 도입)을 도입하여 신경망이 다양한 현상을 학습하는데 도움을 줌

In [12]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1890,  0.2172,  0.2929,  0.2679,  0.5812,  0.5988, -0.1270,  0.2728,
         -0.0144, -0.2092,  0.6068, -0.0863,  0.2053,  0.0428, -0.0362, -0.1855,
          0.4007, -0.1598, -0.0207, -0.0243],
        [-0.3523,  0.3101,  0.1134,  0.3618,  0.4480,  0.2914,  0.1131, -0.0424,
          0.2019, -0.1216,  0.4224, -0.5469, -0.1028, -0.1027, -0.2612, -0.3642,
          0.5678, -0.0731, -0.0752,  0.4183],
        [-0.3766,  0.0958, -0.0521,  0.2631,  0.5142,  0.6418,  0.2586,  0.2227,
         -0.1698,  0.1417,  0.4597, -0.2390, -0.2436,  0.1399, -0.3703, -0.5377,
          0.5746, -0.1733, -0.1355,  0.1572]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2172, 0.2929, 0.2679, 0.5812, 0.5988, 0.0000, 0.2728, 0.0000,
         0.0000, 0.6068, 0.0000, 0.2053, 0.0428, 0.0000, 0.0000, 0.4007, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.3101, 0.1134, 0.3618, 0.4480, 0.2914, 0.1131, 0.0000, 0.2019,
         0.0000, 0.4224, 0.0000, 0.0000, 0.0000, 0.00

## nn.Squential
``nn.Squential`` 은 정렬된 모듈의 컨테이너(container)
 - 데이터는 정의된 것과 동일한 순서로 모든 모듈을 통해 전달됨
 - sequential containers를 사용하여 ``seq_modules``와 같은 빠른 네트워크를 구성할 수 있음

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## nn.Softmax
neural network의 마지막 linear layer는 ``nn.Softmax`` 모듈에 전달되는 logits-raw values in [-infty, infty]-를 return
 - logits는 모델의 예측 확률을 나타내는 값 [0,1]로 조정됨
 - dim parameter는 값의 합이 1이 되어야하는 차원을 나타냄

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters
신경망 내부의 많은 layer는 parameterized 되어 있음.
 - 즉, training 동안 weights and biases(가중치와 편향)을 가짐
 - ``nn.Module``을 subclassing 하면 모델 객체 내에 정의된 모든 필드를 자동으로 추적하고 모델의 model's parameter() 또는 named_pardmater() method를 사용하여 모든 매개변수에 access 할 수 있음

In [15]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0130, -0.0125,  0.0077,  ..., -0.0082,  0.0039,  0.0091],
        [-0.0159, -0.0053, -0.0130,  ..., -0.0245, -0.0237,  0.0223]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-4.8978e-05, -2.9153e-03], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0004, -0.0341, -0.0005,  ..., -0.0185,  0.0428,  0.0409],
        [-0.0341, -0.0068, -0.0069,  ..., -0.0032,  0.0102, -0.0248]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2